<a href="https://colab.research.google.com/github/zerotodeeplearning/masterclass/blob/master/notebooks/Data_Manipulation_with_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Copyright 2020 Catalit LLC.

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Data Manipulation with Pandas

Pandas is an open source library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.

You can find it here: http://pandas.pydata.org/

And the documentation can be found here: https://pandas.pydata.org/docs/getting_started/index.html

In this notebook we review some of its functionality.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

In [ ]:
pd.__version__

In [ ]:
sns.__version__

### Reading data from multiple sources

In [ ]:
url = "https://raw.githubusercontent.com/zerotodeeplearning/ztdl-masterclasses/master/data/"

In [ ]:
#csv
df = pd.read_csv(url + "time_series_covid19_confirmed_global.csv")

In [ ]:
df.head()

In [ ]:
# json
df = pd.read_json(url + "gdp.json")

In [ ]:
df.head()

In [ ]:
#html
dfl = pd.read_html(url + "FDIC_%20Failed%20Bank%20List.htm")

print(type(dfl))
print(len(dfl))

In [ ]:
dfl[0].head()

### Quick exploration

Commands to quickly inspect the dataset

In [ ]:
df = pd.read_csv(url + "titanic-train.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

### Plotting

In [ ]:
survived_counts = df['Survived'].value_counts()
survived_counts

In [ ]:
survived_counts.plot(kind='bar', title='Dead / Survived');

In [ ]:
survived_counts.plot(kind='pie',
            figsize=(5, 5),
            explode=[0, 0.15],
            labels=['Dead', 'Survived'],
            autopct='%1.1f%%',
            shadow=True,
            startangle=90,
            fontsize=16);

In [ ]:
df['Age']

In [ ]:
df['Age'].plot(kind='hist',
               bins=16,
               range=(0, 80),
               title='Passenger age distribution')
plt.xlabel("Age");

In [ ]:
df['Age'].plot(kind='hist',
               bins=80,
               range=(0, 80),
               title='Passenger cumulative age distibution',
               cumulative=True,
               density=True)
plt.xlabel("Age")
plt.axhline(0.25, color='red')
plt.axhline(0.5, color='red')
plt.axhline(0.75, color='red')

### Fancy plotting with seaborn

In [ ]:
sns.pairplot(df[['Age', 'Pclass', 'Fare', 'SibSp', 'Survived']],
             hue='Survived');

In [ ]:
sns.jointplot(x='Age', y='Fare', data=df)

In [ ]:
corr = df.corr()
sns.heatmap(corr);

### Indexing

Retrieving elements by row, by column or both. Try to understand each of the following statements

In [ ]:
df['Ticket']

In [ ]:
df[['Fare', 'Ticket']]

In [ ]:
df.iloc[3]

In [ ]:
df.iloc[3, 4]

In [ ]:
df.loc[0:4,'Ticket']

In [ ]:
df.loc[0:4, ['Fare', 'Ticket']]

### Selections

Retrieving part of the dataframe based on a condition. Try to understand each of the following statements.

In [ ]:
df[df.Age > 70]

In [ ]:
df[(df['Age'] == 11) & (df['SibSp'] == 5)]

In [ ]:
df[(df.Age == 11) | (df.SibSp == 5)]

In [ ]:
df.query('(Age == 11) & (SibSp == 5)')

In [ ]:
df.sort_values('Age', ascending = False).head()

### Distinct elements

In [ ]:
df['Embarked'].unique()

### Group by

Pandas supports many SQL-like operations like group by, order by and join. In pandas they are called:
- groupby
- sort_values
- merge

Try to make sense of each of the following expressions:

In [ ]:
# Find average age of passengers that survived vs. died
df.groupby('Survived')['Age'].mean()

In [ ]:
df.sort_values('Age', ascending = False).head()

### Join (merge)

In [ ]:
df1 = df[['PassengerId', 'Survived']]
df2 = df[['PassengerId', 'Age']]

pd.merge(df1, df2, on='PassengerId').head()

### Pivot Tables

Pandas also supports Excel-like functionality like pivot tables.

In [ ]:
df.pivot_table(index='Pclass', columns='Survived', values='PassengerId', aggfunc='count')

In [ ]:
df['Pclass'].value_counts()

### Exercise 1:

Try doing a few of these:

- select passengers that survived
- select passengers that embarked in port S
- select male passengers
- select passengers who paid less than 40.000 and were in third class
- locate the name of passegner Id 674
- calculate the average age of passengers using the function mean()
- count the number of survived and the number of dead passengers
- count the number of males and females
- count the number of survived and dead per each gender
- calculate average price paid by survived and dead people in each class

### Time series data

In [ ]:
df = pd.read_csv(url + "time_series_covid19_confirmed_global.csv")

In [ ]:
df.head()

### Dropping columns

In [ ]:
df1 = df.drop(['Lat', 'Long'], axis=1)
df1

### Groupby + Aggregation

In [ ]:
dfcountries = df1.groupby('Country/Region').sum().transpose()
dfcountries

### Data Time index

In [ ]:
dfcountries.index

In [ ]:
dfcountries.index = pd.to_datetime(dfcountries.index)

In [ ]:
dfcountries.index

In [ ]:
dfcountries.head()

In [ ]:
series = dfcountries['Italy']
series[series > 100].plot(style='o-', logy=True, title='Italy')
plt.ylabel('Confirmed Cases');

In [ ]:
country = 'Italy'
ts = dfcountries[country].diff()
ts = ts[ts>0]


fig, ax = plt.subplots(figsize=(10,6))
ax.bar(ts.index, ts.values)
ax.xaxis.set_major_locator(mdates.WeekdayLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
ax.set_title(f'{country}: New cases')
ax.set_ylabel('Count')
ax.set_xlabel('Date');

### Exercise 2:

- find the top 20 countries by number of cases. You will need to use selections and these methods:
    - `.tail()`
    - `.transpose`
    - `.sort_values`

    Your goal is to obtain a Pandas Series that looks like this:
  ```
  Country/Region
  US                188172
  Italy             105792
  Spain              95923
  China              82279
  Germany            71808
  France             52827
  ...
   ```
- use a horizontal bar plot to plot them
- bonus points if you add title and labels